In [ ]:
!pip install Pypdf
! pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
import gensim
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim import corpora
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_url(url):
    # Send an HTTP request to fetch the page content
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the relevant content (e.g., paragraphs)
    paragraphs = soup.find_all("p")
    return [p.get_text() for p in paragraphs]

urls = [
    "https://thehackernews.com/2024/07/how-searchable-encryption-changes-data.html",
    "https://thehackernews.com/2022/08/conti-cybercrime-cartel-using-bazarcall.html",
    "https://thehackernews.com/2024/06/moreeggs-malware-disguised-as-resumes.html",
    "https://thehackernews.com/2022/10/bazarcall-callback-phishing-attacks.html",
    "https://thehackernews.com/2024/05/mysterious-cyber-attack-takes-down.html",
    "https://thehackernews.com/2023/05/new-phishing-as-service-platform-lets.html",
    "https://thehackernews.com/2024/03/apt28-hacker-group-targeting-europe.html"
]

text_data = []
for url in urls:
    text_data.extend(fetch_url(url))
  # Create a Pandas DataFrame
df = pd.DataFrame({"Text": text_data})

# Print the DataFrame
print(df)

                                                  Text
0    Searchable Encryption has long been a mystery....
1    Organizations know they must encrypt their mos...
2    The best they could do was to wrap that plaint...
3    It's safe to conclude that the way we're secur...
4                                                     
..                                                 ...
135  The climax of APT28's elaborate scheme ends wi...
136  "ITG05 remains adaptable to changes in opportu...
137  Guard your business like a Fortune 500 with a ...
138  Get actionable steps and tools to harness the ...
139  Get the latest news, expert insights, exclusiv...

[140 rows x 1 columns]


In [ ]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Basic Preprocessing
data = df['Text'].str.lower()
df["Text"]=data.apply(lambda s: ' '.join(re.sub("[.,!?:;-='*$^...'@#_/|&]", " ", s).split()))
df["Text"] = df["Text"].str.replace(r'\d+', '', regex=True)
punctuations = string.punctuation
df["Text"] = df["Text"].apply(lambda x: x.translate(str.maketrans('', '', punctuations)))
df.head()

,Text
0,searchable encryption has long been a mystery ...
1,organizations know they must encrypt their mos...
2,the best they could do was to wrap that plaint...
3,it s safe to conclude that the way we re secur...
4,


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    return ' '.join(lemmatized_words)

Text = df['Text'].apply(preprocess_text)

print(Text)

0      searchable encryption long mystery oxymoron un...
1      organization know must encrypt valuable sensit...
2      best could wrap plaintext unencrypted data wit...
3      safe conclude way securing data working critic...
4                                                       
                             ...                        
135    climax apt elaborate scheme end execution mase...
136    itg remains adaptable change opportunity deliv...
137    guard business like fortune fraction resource ...
138    get actionable step tool harness full potentia...
139    get latest news expert insight exclusive resou...
Name: Text, Length: 140, dtype: object


In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(Text )

# Calculate cosine similarity
scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
# Select the top n sentences with the highest scores
n=15
summary_sentences = nlargest(n, range(len(scores)), key=scores.__getitem__)

summary_tfidf = ' '.join([Text[i] for i in sorted(summary_sentences)])

summary_tfidf

'brings another layer complexity—the expertise dba database expert time decrypt build security enclave complex solution designed monitor secure plaintext datastore enclave complex solution requires specialized team expert knowledge security tool function add need patch refresh security tool maintain effectiveness understand much data compromised daily guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free attack unsuccessful targeted unnamed company industrial service industry may canadian cybersecurity firm esentire disclosed last week guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free guard business like for

In [ ]:
summary = summary_tfidf
summary_sentences = summary.split('. ')
formatted_summary = '.\n'.join(summary_sentences)

print(formatted_summary)

brings another layer complexity—the expertise dba database expert time decrypt build security enclave complex solution designed monitor secure plaintext datastore enclave complex solution requires specialized team expert knowledge security tool function add need patch refresh security tool maintain effectiveness understand much data compromised daily guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free attack unsuccessful targeted unnamed company industrial service industry may canadian cybersecurity firm esentire disclosed last week guard business like fortune fraction resource find allinone solution gamechanger get latest news expert insight exclusive resource strategy industry leader – free guard business like fort

In [ ]:
reference_summary="As stated, it's well established that most data is not being encrypted. Just look at the well documented, ongoing growth rate of cybercrime activity. In short, all data breaches and data ransom cases have one glaring common thread— every target maintains millions of private, sensitive, and confidential records in an unencrypted state. Stores of data, fully indexed, structured and unencrypted as easy to read plaintext simply to support operational use cases. This challenge falls under the auspices of Acceptable Risk.It's often viewed that if an organization has good cyber hygiene, that organization is encrypting data at rest (in storage, archived, or backed up) and in transit or motion (i.e. email encryption, or sending data from one point to another point). And many may think that's enough—or that is the best they can do. After all, encryption at rest and in motion is the only encryption focus of current compliance and governance bodies, where they address database encryption"

In [ ]:
# Evaluation
import rouge
from rouge import Rouge
def evaluate_rouge(reference_text, summary_text):
  rouge = Rouge()
  scores = rouge.get_scores(reference_text, summary_text)
  return scores[0]['rouge-1']['f']

In [ ]:
summary

# Evaluate the summary using ROUGE
rouge_score = evaluate_rouge(reference_summary, summary)

print(f"ROUGE score: {rouge_score}")

ROUGE score: 0.05291004821925519


Abstractive Summarisation

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model=T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer=T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_text = "In today's cyber-centric business landscape, the need for searchable encryption spans many industries and use cases such as Financial Services, Healthcare, Banking, Manufacturing, Government, Education, Critical Infrastructure, Retail, and Research to name a few. There isn't an area where data doesn't need to be more SAFE."
input=tokenizer.encode(input_text,return_tensors='pt',max_length=512,truncation=True)

In [ ]:
output=model.generate(input,max_length=150,num_beams=7,early_stopping=True)

In [ ]:
output_text=tokenizer.decode(output[0],skip_special_tokens=True)
print(output_text)

The need for searchable encryption spans many industries and use cases such as Financial Services, Healthcare, Banking, Manufacturing, Government, Education, Critical Infrastructure, Retail, and Research.
